<a href="https://colab.research.google.com/github/eleanarey/ProgramingPractices/blob/main/ReyQuijadaEleanaLiscarCAT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the moment of writing this document, Cuda toolkit is already installed in the Colab
environment (in previous semesters it was not the case, so we need to install it manually).
We can check the compiler version running the following command within a cell:

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


The first line of the cell executes the Linux command that set up the software requirements in the underlying operating system of the host machine that runs the Jupyter environment. The second line loads the CUDA environment in the Jupyter notebook:


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5_vesuil
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5_vesuil
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=a970e43e8b3282005b92a3e656462936fde0268048f05c898c8887e184fa722d
  Stored in directory: /tmp/pip-ephem-wheel-cache-aspfjpu4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


Once it is finished, we will be able to run the CUDA C/C++ code using the extension
%%cu at the beginning of each cell.
For instance, this code implements the typical “hello world”:

In [3]:
%%cu
#include <stdio.h>

__global__ void hello_kernel(void) {
    printf("Hello world from the device!\n");
}

int main(void) {
    printf("Hello world from the host!\n");
    hello_kernel<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello world from the host!
Hello world from the device!



Threading: If we know the problem size and the block size, we could calculate the number of blocks.

### 1. Provide the code that generates an output similar to this: For having the maximum mark in this exercise, you have to explain every implementation decision:

- Host and Device Code Separation:
The CUDA programming model consists of host (CPU) and device (GPU) code. The main function runs on the host and launches kernels which run on the device. This separation is crucial for managing computations that are offloaded to the GPU.

- Kernel Design (printThreadIds):
The kernel is designed to be lightweight and autonomous. Each instance (thread) executes the same code but operates on different data, following the SIMT (Single Instruction, Multiple Thread) architecture. This design ensures efficient parallel execution where each thread knows its unique position in the thread grid.

- Global ID Calculation:
The global ID for each thread is calculated using blockId * blockSize + threadId. This formula ensures a unique ID across the entire grid. It's a standard approach in CUDA for identifying threads when they need to work on different parts of an array or dataset.

- Use of Built-in Variables (blockIdx, threadIdx, blockDim):
These are CUDA built-in variables that provide each thread with its context within the grid and block. They are essential for determining the thread's position and for computing its global ID.

- Kernel Launch Configuration:
The numBlocks and blockSize variables define the execution configuration. The choice of 5 for both is based on the output requirement, demonstrating an understanding of how to map problem dimensions to the CUDA grid hierarchy.

- Use of printf in Kernel:
CUDA supports a limited use of printf within kernel code for debugging purposes. It's used here to directly output each thread's information to the standard output on the host. This is for demonstration and learning purposes; in a production environment, you would typically avoid I/O operations within kernels.

- Synchronization with cudaDeviceSynchronize:
This function is used to synchronize the host and device, ensuring that all kernel executions are completed before the host continues execution. It's essential for correctness when the host needs to interact with data that the device has processed.

- Error Checking (to be implemented in a complete solution):
While not included in the provided snippet, proper error checking after each CUDA API call and kernel launch is critical for robustness and correctness. It allows for the detection and handling of runtime errors, such as failed kernel launches or issues with memory allocation.

- Resource Management (to be implemented in a complete solution):
Deallocating any dynamically allocated memory on the device and resetting the device at the end of the program are best practices that prevent resource leaks and ensure a clean state for subsequent CUDA operations.

In [4]:
%%cu
#include <stdio.h>

// CUDA Kernel function to print thread IDs
__global__ void printThreadIds() {
    int blockId = blockIdx.x;
    int threadId = threadIdx.x;
    int blockSize = blockDim.x;

    // Calculate global ID
    int globalId = blockId * blockSize + threadId;

    // Print the message
    printf("Hi! My Id is %d, I am the thread %d out of %d in block %d\n", globalId, threadId, blockSize, blockId);
}

int main() {
    // Define the number of blocks and threads per block
    int numBlocks = 5;
    int blockSize = 5; // This means each block contains 5 threads

    // Launch the kernel
    printThreadIds<<<numBlocks, blockSize>>>();

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    return 0;
}


Hi! My Id is 20, I am the thread 0 out of 5 in block 4
Hi! My Id is 21, I am the thread 1 out of 5 in block 4
Hi! My Id is 22, I am the thread 2 out of 5 in block 4
Hi! My Id is 23, I am the thread 3 out of 5 in block 4
Hi! My Id is 24, I am the thread 4 out of 5 in block 4
Hi! My Id is 5, I am the thread 0 out of 5 in block 1
Hi! My Id is 6, I am the thread 1 out of 5 in block 1
Hi! My Id is 7, I am the thread 2 out of 5 in block 1
Hi! My Id is 8, I am the thread 3 out of 5 in block 1
Hi! My Id is 9, I am the thread 4 out of 5 in block 1
Hi! My Id is 15, I am the thread 0 out of 5 in block 3
Hi! My Id is 16, I am the thread 1 out of 5 in block 3
Hi! My Id is 17, I am the thread 2 out of 5 in block 3
Hi! My Id is 18, I am the thread 3 out of 5 in block 3
Hi! My Id is 19, I am the thread 4 out of 5 in block 3
Hi! My Id is 10, I am the thread 0 out of 5 in block 2
Hi! My Id is 11, I am the thread 1 out of 5 in block 2
Hi! My Id is 12, I am the thread 2 out of 5 in block 2
Hi! My Id is 13

Memory Allocation

### 2.Regarding the code:

• The code does not work properly. What have you done to
correct it?
- Add cudaMemcpy calls before kernel execution to transfer input data from host to device memory. Ensure that device memory is freed after the data is copied back to the host to avoid memory leaks.

• Change the value of “BLOCKSIZE” to, for instance, “3”.
How does it affect the execution compared to the original
output?
 - The kernel will now be launched with fewer threads per block (BLOCKSIZE = 3). This means that in each thread block, only three threads will be active, and since the grid size is 1, only three characters of the string will be modified.

- If BLOCKSIZE is less than N, not all elements of a and b will be processed, leading to an incomplete operation. In this case, with BLOCKSIZE = 3, only the first three characters of the string will be modified, and the rest will remain unchanged.
In summary, when modifying BLOCKSIZE, it's crucial to ensure that it matches the size of the data being processed to achieve the desired computation across the entire dataset. If there are more data elements than threads, some data will not be processed unless additional thread blocks are added to the grid.

In [5]:
%%cu
#include <stdio.h>

const int N = 16;
const int GRIDSIZE = 1; //number of thread blocks
const int BLOCKSIZE = 32; //number of threads per thread block

__global__ void hello_decoder(char *a, int *b) {
    a[threadIdx.x] += b[threadIdx.x];
}

int main() {
    char a[N] = "Hello\0\0\0\0\0\0";
    int b[N] = {15, 10, 6, 0, -11, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0};
    char *ad;
    int *bd;
    const int csize = N*sizeof(char);
    const int isize = N*sizeof(int);

    printf("%s ", a);

    cudaMalloc((void**)&ad, csize);
    cudaMalloc((void**)&bd, isize);

    // Copy input data from host to device
    cudaMemcpy(ad, a, csize, cudaMemcpyHostToDevice);
    cudaMemcpy(bd, b, isize, cudaMemcpyHostToDevice);

    // Kernel launch
    hello_decoder<<<GRIDSIZE, BLOCKSIZE>>>(ad, bd);

    // Copy output data from device to host
    cudaMemcpy(a, ad, csize, cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(ad);
    cudaFree(bd);

    printf("%s\n", a);

    return EXIT_SUCCESS;
}


Hello World



### 3. Provide the kernel code that solves the problem and answer the following questions:

__global__ void add(float *x, float *y) {
 int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
    for (int i = index; i < PROBLEMSIZE; i += stride) {
        y[i] += x[i];
    }
}

###• How different is managed transfers between CPU and GPU?

Managed memory (cudaMallocManaged) simplifies memory management by providing a single memory space accessible from both CPU and GPU. It allows for automatic data migration between the host and device, eliminating the need for explicit cudaMemcpy calls. However, this can lead to performance overhead due to on-demand paging. In contrast, explicit memory transfers require the programmer to manage separate memory spaces and perform cudaMemcpy operations to move data between host and device.

###• Check that it does not return an error (you can attach a screenshot).

After the kernel execution, the code checks for errors by verifying that the sum in array y is equal to VALUE. If there is any difference, it prints an error message.

###• How long does it take to run (you can use the extension %%time at the beginning of the cell, or the Unix command time before the binary execution)?
The actual time taken depend on the GPU's capabilities and the current load on the system. For large problem sizes, such as PROBLEMSIZE = 1000000000, the execution could take a significant amount of time.
-Finally the time was:

*real	0m18.612s

*user	0m14.640s

*sys	0m3.229s

In [6]:

%%writefile exercise.cu
#include <iostream>
#include <math.h>
#define VALUE 20
#define PROBLEMSIZE 1000000000

__global__ void add(float *x, float *y) {
 int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
    for (int i = index; i < PROBLEMSIZE; i += stride) {
        y[i] += x[i];
    }
}

int main(void) {
    float *x, *y;
    cudaMallocManaged(&x, PROBLEMSIZE * sizeof(float));
    cudaMallocManaged(&y, PROBLEMSIZE * sizeof(float));
    for (int i = 0; i < PROBLEMSIZE; i++) {
        float val = (float)(i % VALUE);
        x[i] = val;
        y[i] = (VALUE - val);
    }

    int blockSize = 256;
    int numBlocks = (PROBLEMSIZE + blockSize - 1) / blockSize;
    add<<<numBlocks, blockSize>>>(x, y);
    cudaDeviceSynchronize();


    float error = 0.0f;
    for (int i = 0; i < PROBLEMSIZE; i++)
        error = fmax(error, fabs(y[i] - VALUE));
    if (error != 0)
        printf("Wrong result. Check your code, especially your kernel\n");

    cudaFree(x);
    cudaFree(y);
    return 0;
}


Overwriting exercise.cu


In [9]:
! nvcc --version
! nvcc -o exercise exercise.cu
! time ./exercise

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0

real	0m18.612s
user	0m14.640s
sys	0m3.229s


### 4. If you have not changed the number of blocks and threads, probably you are aware that this code is not leveraging the massively parallel architecture of the GPU.

###• Provide block/thread configurations that reduces the execution time. Compare the configurations and the execution time (for instance, with a table).
We can optimize the execution of the CUDA kernel by adjusting the block and thread configurations to better utilize the GPU's parallel architecture. The goal is to increase the occupancy of the GPU by launching a sufficient number of blocks and threads so that while some threads are waiting for data to arrive from memory (memory latency).

Original Configuration:
blockSize = 256: This means each block has 256 threads.
numBlocks: It is calculated to cover the entire PROBLEMSIZE, ensuring all elements are processed.

Optimization Strategy:
Increase the Grid Size: The number of blocks (gridDim.x) should be increased to ensure that there are enough blocks to utilize all the SMs on the GPU. For the given figure, gridDim.x is 1024, which indicates the maximum number of blocks along the x-dimension that can be launched.

Modified Kernel Launch:
int blockSize = 64;
int numBlocks = min(1024, (PROBLEMSIZE + blockSize - 1) / blockSize);
//min is used to ensure that we do not launch more blocks than the GPU supports. The 1024 value comes from the GPU's capability as per the figure provided.
add<<<numBlocks, blockSize>>>(x, y);

###• Is there any limitation on the blocks or threads quantities? Hint: You may check again Figure 5.
Thread Configuration: Based on the figure, blockDim.x is 64, suggesting that each block can have up to 64 threads.

Calculate Index and Stride: The index calculation doesn't change, but we ensure that it reflects the updated block and thread configuration. The stride is the total number of threads in the grid, which is blockDim.x * gridDim.x.

Maximum Threads per Block: Typically, this is 1024 for most modern GPUs.
Maximum Blocks per Grid: This is often 2^31 - 1 for each grid dimension but can be lower based on GPU architecture.
Maximum Threads per SM: This is hardware-dependent and can be found in the GPU's specification.
Shared Memory and Registers: More threads may require more shared memory and registers, which are limited.


In [10]:
%%writefile optimization.cu
#include <iostream>
#include <math.h>
#define VALUE 20
#define PROBLEMSIZE 1000000000

__global__ void add(float *x, float *y) {
 int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
    for (int i = index; i < PROBLEMSIZE; i += stride) {
        y[i] += x[i];
    }
}

int main(void) {
    float *x, *y;
    cudaMallocManaged(&x, PROBLEMSIZE * sizeof(float));
    cudaMallocManaged(&y, PROBLEMSIZE * sizeof(float));
    for (int i = 0; i < PROBLEMSIZE; i++) {
        float val = (float)(i % VALUE);
        x[i] = val;
        y[i] = (VALUE - val);
    }


    // start optimization
    int blockSize = 64; // Number of threads per block
    int numBlocks = min(1024, (PROBLEMSIZE + blockSize - 1) / blockSize); // Number of blocks
    add<<<numBlocks, blockSize>>>(x, y);
    cudaDeviceSynchronize();
    // end optimization



    float error = 0.0f;
    for (int i = 0; i < PROBLEMSIZE; i++)
        error = fmax(error, fabs(y[i] - VALUE));
    if (error != 0)
        printf("Wrong result. Check your code, especially your kernel\n");

    cudaFree(x);
    cudaFree(y);
    return 0;
}


Writing optimization.cu


In [27]:
! nvcc --version
! nvcc -o optimization optimization.cu
! time ./optimization

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0

real	0m19.535s
user	0m15.703s
sys	0m3.548s


In [29]:
! nvcc  -o optimization optimization.cu
! nvprof ./optimization
! time ./optimization

==42532== NVPROF is profiling process 42532, command: ./optimization
==42532== Profiling application: ./optimization
==42532== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.29169s         1  3.29169s  3.29169s  3.29169s  add(float*, float*)
      API calls:   83.25%  3.29170s         1  3.29170s  3.29170s  3.29170s  cudaDeviceSynchronize
                   11.50%  454.77ms         2  227.38ms  188.22ms  266.55ms  cudaFree
                    5.24%  207.08ms         2  103.54ms  67.372us  207.01ms  cudaMallocManaged
                    0.01%  300.53us         1  300.53us  300.53us  300.53us  cudaLaunchKernel
                    0.00%  153.68us       114  1.3480us     145ns  60.834us  cuDeviceGetAttribute
                    0.00%  13.976us         1  13.976us  13.976us  13.976us  cuDeviceGetName
                    0.00%  5.6440us         1  5.6440us  5.6440us  5.6440us  cuDeviceTotalMem
                 

In [18]:
%%writefile optimization2.cu
#include <iostream>
#include <math.h>
#define VALUE 20
#define PROBLEMSIZE 1000000000

__global__ void add(float *x, float *y) {
 int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
    for (int i = index; i < PROBLEMSIZE; i += stride) {
        y[i] += x[i];
    }
}

int main(void) {
    float *x, *y;
    cudaMallocManaged(&x, PROBLEMSIZE * sizeof(float));
    cudaMallocManaged(&y, PROBLEMSIZE * sizeof(float));
    for (int i = 0; i < PROBLEMSIZE; i++) {
        float val = (float)(i % VALUE);
        x[i] = val;
        y[i] = (VALUE - val);
    }


    // start optimization
    int blockSize = 256; // Number of threads per block
    int numBlocks = (PROBLEMSIZE + blockSize - 1) / blockSize; // Number of blocks in the grid
    add<<<numBlocks, blockSize>>>(x, y);
    cudaDeviceSynchronize();
    // end optimization



    float error = 0.0f;
    for (int i = 0; i < PROBLEMSIZE; i++)
        error = fmax(error, fabs(y[i] - VALUE));
    if (error != 0)
        printf("Wrong result. Check your code, especially your kernel\n");

    cudaFree(x);
    cudaFree(y);
    return 0;
}


Overwriting optimization2.cu


###5. Provide the code of the non-managed CPU-GPU memory version of the problem.

###• Which are the differences in the dominant API calls when the block size is equal to “1” when comparing managed and non-managed versions? Why?

Managed Version: Uses cudaMallocManaged() for memory accessible by both the CPU and GPU, simplifying data transfer.

Unmanaged Version: Requires explicit memory allocation and transfer between CPU and GPU using cudaMalloc() and cudaMemcpy().

Impact on Performance:
Both versions suffer from limited performance with a block size of "1," not leveraging the GPU's parallel capability.

Managed Version: May have additional overhead due to automatic memory management.

Unmanaged Version: Offers explicit control over memory but is still limited by low parallelism.

Why?
Low utilization of the GPU's parallel resources affects both versions.
Automatic memory management in the managed version does not compensate for the low parallel utilization.

The unmanaged version, while more predictable in memory management, also faces limitations of parallelism.

In summary, the main difference lies in memory management, but both versions have limited performance due to low use of the GPU's parallelism.

In [38]:
%%writefile non_managed.cu
#include <iostream>
#include <math.h>
#define VALUE 20
#define PROBLEMSIZE 1000000000

__global__ void add(float *x, float *y) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < PROBLEMSIZE; i += stride) {
    y[i] += x[i];
  }
}

int main(void) {
  float *x, *y;
  float *d_x, *d_y; // Device pointers
  size_t size = static_cast<size_t>(PROBLEMSIZE) * sizeof(float); // Corrected size type

  // Allocate host memory
  x = (float*)malloc(size);
  y = (float*)malloc(size);
  if (x == nullptr || y == nullptr) {
    std::cerr << "Failed to allocate host memory!" << std::endl;
    return -1;
  }

  // Initialize host arrays
  for (int i = 0; i < PROBLEMSIZE; i++) {
    float val = static_cast<float>(i % VALUE);
    x[i] = val;
    y[i] = (VALUE - val);
  }

  // Allocate device memory
  cudaMalloc(&d_x, size);
  cudaMalloc(&d_y, size);

  // Copy data from host to device
  cudaMemcpy(d_x, x, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, size, cudaMemcpyHostToDevice);

  // Execute the kernel
  int blockSize = 5000;
  int numBlocks = (PROBLEMSIZE + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(d_x, d_y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Copy data back from device to host
  cudaMemcpy(y, d_y, size, cudaMemcpyDeviceToHost);

  // Check for errors (all values should be 20.0f)
  float error = 0.0f;
  for (int i = 0; i < PROBLEMSIZE; i++)
    error = fmax(error, fabs(y[i] - VALUE));
  if (error != 0)
    printf("Wrong result. Check your code, especially your kernel\n");

  // Free memory
  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);

  return 0;
}

Overwriting non_managed.cu


In [39]:
! nvcc -o non_managed non_managed.cu
! time ./non_managed


Wrong result. Check your code, especially your kernel

real	0m18.685s
user	0m14.191s
sys	0m4.423s


In [34]:
! nvprof ./non_managed
! time ./non_managed

==55249== NVPROF is profiling process 55249, command: ./non_managed
==55249== Profiling application: ./non_managed
==55249== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.83%  1.69949s         2  849.74ms  848.32ms  851.17ms  [CUDA memcpy HtoD]
                   34.45%  917.31ms         1  917.31ms  917.31ms  917.31ms  [CUDA memcpy DtoH]
                    1.72%  45.725ms         1  45.725ms  45.725ms  45.725ms  add(float*, float*)
      API calls:   91.41%  2.61766s         3  872.55ms  848.53ms  917.71ms  cudaMemcpy
                    6.70%  191.90ms         2  95.951ms  3.4305ms  188.47ms  cudaMalloc
                    1.60%  45.735ms         1  45.735ms  45.735ms  45.735ms  cudaDeviceSynchronize
                    0.27%  7.8064ms         2  3.9032ms  2.9312ms  4.8752ms  cudaFree
                    0.01%  230.26us         1  230.26us  230.26us  230.26us  cudaLaunchKernel
                    0.00%  141.